Which Legendre Polynomials Should We Use
------

I am having trouble constructing the constraint matrix on the diagonal face. The top and bottom are easy, that is done. The question is: which tensor product Legendre polynomials do I use on the diagonal face?

In [16]:
from sympy import *
import numpy as np
x,y,z,s,t = symbols('x y z s t')

First I will define the 2nd and 3rd order tensor product Legendre polynomials in $s,z$, the coordinates of the diagonal plane.

In [17]:
L = []
L.append(legendre(2,s)*legendre(0,z)) # n = 2
L.append(legendre(1,s)*legendre(1,z))
L.append(legendre(0,s)*legendre(2,z))
L.append(legendre(3,s)*legendre(0,z)) # n = 3
L.append(legendre(2,s)*legendre(1,z))
L.append(legendre(1,s)*legendre(2,z))
L.append(legendre(0,s)*legendre(3,z))

Then I will define the full hexahedral basis as in (Wheeler, Xue, & Yotov, 2012) and use it as a prime basis $P$.

In [15]:
def curl(v):
    c = []
    c.append(  diff(v[2],y)-diff(v[1],z) )
    c.append(-(diff(v[2],x)-diff(v[0],z)))
    c.append(  diff(v[1],x)-diff(v[0],y) )
    return np.asarray(c)

P = np.asarray([[1,0,0],
                [x,0,0],
                [y,0,0],
                [z,0,0],
                [0,1,0],
                [0,x,0],
                [0,y,0],
                [0,z,0],
                [0,0,1],
                [0,0,x],
                [0,0,y],
                [0,0,z]])
P = np.vstack([P,curl([0       ,0       ,x*y*z   ])])
P = np.vstack([P,curl([0       ,0       ,x*y**2  ])])
P = np.vstack([P,curl([0       ,0       ,x**2*z  ])])
P = np.vstack([P,curl([0       ,0       ,x**2*y*z])])
P = np.vstack([P,curl([x*y*z   ,0       ,0       ])])
P = np.vstack([P,curl([y*z**2  ,0       ,0       ])])
P = np.vstack([P,curl([x*y**2  ,0       ,0       ])])
P = np.vstack([P,curl([x*y**2*z,0       ,0       ])])
P = np.vstack([P,curl([0       ,x*y*z   ,0       ])])
P = np.vstack([P,curl([0       ,x**2*z  ,0       ])])
P = np.vstack([P,curl([0       ,y*z**2  ,0       ])])
P = np.vstack([P,curl([0       ,x*y*z**2,0       ])])

for i in range(P.shape[0]):
    print("P[%2d] = [%10s, %10s, %10s]" % (i,P[i,0],P[i,1],P[i,2]))

P[ 0] = [         1,          0,          0]
P[ 1] = [         x,          0,          0]
P[ 2] = [         y,          0,          0]
P[ 3] = [         z,          0,          0]
P[ 4] = [         0,          1,          0]
P[ 5] = [         0,          x,          0]
P[ 6] = [         0,          y,          0]
P[ 7] = [         0,          z,          0]
P[ 8] = [         0,          0,          1]
P[ 9] = [         0,          0,          x]
P[10] = [         0,          0,          y]
P[11] = [         0,          0,          z]
P[12] = [       x*z,       -y*z,          0]
P[13] = [     2*x*y,      -y**2,          0]
P[14] = [         0,     -2*x*z,          0]
P[15] = [    x**2*z,   -2*x*y*z,          0]
P[16] = [         0,        x*y,       -x*z]
P[17] = [         0,      2*y*z,      -z**2]
P[18] = [         0,          0,     -2*x*y]
P[19] = [         0,     x*y**2,   -2*x*y*z]
P[20] = [      -x*y,          0,        y*z]
P[21] = [     -x**2,          0,      2*x*z]
P[22] = [ 

Again, we already know how to remove $xy$ from the top and bottom faces. The question is: what do we need to remove from the diagonal face and how do we accomplish that? So to see what spaces are spanned on that face, we apply a rotation to local coordinates $(x,y,z)\rightarrow(s,t,z)$ where $s,z$ are two orthogonal directions tangent to the face and $t$ is normal to the face. Then we evaluate $t=0$ and dot by the outward $t$ normal to see the prime basis restricted to this plane.

In [19]:
Pr = P.copy()
for i in range(Pr.shape[0]):
    for j in range(Pr.shape[1]):
        if type(Pr[i,j]) == type(0): continue
        Pr[i,j] = expand(P[i,j].subs({x:s-t,y:s+t})) # rotate
diag = np.dot(Pr,[0,1,0]) # positive 't' direction
for i in range(diag.size):
    if type(diag[i]) != type(0): diag[i] = diag[i].subs({t:0})
    print("%2d %18s" % (i,diag[i]))

 0                  0
 1                  0
 2                  0
 3                  0
 4                  1
 5                  s
 6                  s
 7                  z
 8                  0
 9                  0
10                  0
11                  0
12               -s*z
13              -s**2
14             -2*s*z
15          -2*s**2*z
16               s**2
17              2*s*z
18                  0
19               s**3
20                  0
21                  0
22                  0
23                  0


So, as I have noticed before, we see $s^2,s^2z,s^3$ terms in bases 13,15,16,19. As I understand, our goal is to find constraints such that the span on this face is $1,s,z,sz$. So I loop through the 2nd and 3rd order Legendre polynomials for a check on what is orthogonal to these terms.

In [24]:
ind = []
for i in [13,15,16,19]:
    for j in range(len(L)):
        I = integrate(integrate(diag[i]*L[j],(s,-1,1)),(z,-1,1))
        print("%d %d integral[(%s) (%s)] = (%s)" % (i,j,diag[i],L[j],I))
        if I == 0: continue  # orthogonal        
        if j not in ind: ind.append(j)
    print(" ")
print("The following set of Legendre polynomials are needed:")
for i in ind:
    print(i,L[i])

13 0 integral[(-s**2) (3*s**2/2 - 1/2)] = (-8/15)
13 1 integral[(-s**2) (s*z)] = (0)
13 2 integral[(-s**2) (3*z**2/2 - 1/2)] = (0)
13 3 integral[(-s**2) (5*s**3/2 - 3*s/2)] = (0)
13 4 integral[(-s**2) (z*(3*s**2/2 - 1/2))] = (0)
13 5 integral[(-s**2) (s*(3*z**2/2 - 1/2))] = (0)
13 6 integral[(-s**2) (5*z**3/2 - 3*z/2)] = (0)
 
15 0 integral[(-2*s**2*z) (3*s**2/2 - 1/2)] = (0)
15 1 integral[(-2*s**2*z) (s*z)] = (0)
15 2 integral[(-2*s**2*z) (3*z**2/2 - 1/2)] = (0)
15 3 integral[(-2*s**2*z) (5*s**3/2 - 3*s/2)] = (0)
15 4 integral[(-2*s**2*z) (z*(3*s**2/2 - 1/2))] = (-16/45)
15 5 integral[(-2*s**2*z) (s*(3*z**2/2 - 1/2))] = (0)
15 6 integral[(-2*s**2*z) (5*z**3/2 - 3*z/2)] = (0)
 
16 0 integral[(s**2) (3*s**2/2 - 1/2)] = (8/15)
16 1 integral[(s**2) (s*z)] = (0)
16 2 integral[(s**2) (3*z**2/2 - 1/2)] = (0)
16 3 integral[(s**2) (5*s**3/2 - 3*s/2)] = (0)
16 4 integral[(s**2) (z*(3*s**2/2 - 1/2))] = (0)
16 5 integral[(s**2) (s*(3*z**2/2 - 1/2))] = (0)
16 6 integral[(s**2) (5*z**3/2 - 3*z/2)] 

We have 3 terms to remove and it requires just 3 functions, but we need 1 more restriction or the SVD will have a 0 eigenvalue.